In [1]:
# 環境設置
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.keys import Keys  # 鍵盤操作
from selenium.common.exceptions import NoSuchElementException, TimeoutException
# ---------- 等待頁面加載完成
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
# ----------
import pandas as pd
import pandas.io.formats.excel
import requests
import requests.packages.urllib3
requests.packages.urllib3.disable_warnings()
from urllib.request import HTTPError
from bs4 import BeautifulSoup
import time

In [2]:
# webpac_gov_crawler
def click_more_btn(driver):
    try:
        while True:
            time.sleep(2)  # 不得已的強制等待
            driver.find_element_by_css_selector("a.btnstyle.bluebg3.morewidth").click()
    except:
        pass 

# 宜蘭縣公共圖書館、桃園市立圖書館
def webpac_gov_crawler(org, url, ISBN, driver, wait):
    try:
        # 進入《館藏系統》頁面
        driver.get(url)

        # 等待＂進階查詢的按鈕＂直到出現：click
        wait.until(EC.element_to_be_clickable((By.XPATH, '//*[@id="center"]/div/section[1]/div/div/div/form/div[3]/div[1]/a'))).click()

        # 等待＂下拉式選單＂直到出現：選擇以＂ISBN＂方式搜尋，並再搜尋欄位輸入＂ISBN＂後再 submit
        select = Select(wait.until(EC.presence_of_element_located((By.NAME, "searchField"))))
        time.sleep(0.2)  # 瀏覽器反應不過來 QQ
        select.select_by_visible_text(u"ISBN")
        search_input = driver.find_element_by_name("searchInput")
        search_input.send_keys(ISBN)
        search_input.submit()  # 有些網站無法 submit，必須使用 send_keys(Keys.ENTER)

        # 計算搜尋結果的數量（results）
        # 以＂宜蘭縣公共圖書館＂為例，當搜尋結果只有一筆時，網頁會迅速地自動跳轉至＂詳細書目＂
        results = len(wait.until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, "div.booklist"))))
        soup = BeautifulSoup(driver.page_source, "html.parser")  # 用以當搜尋結果多筆時，找尋不同＂詳細書目＂的網址

        # 依照搜尋結果的數量分成：一筆和多筆兩種情形
#         time.sleep(2)
        if results < 2:
            click_more_btn(driver)
            return pd.read_html(driver.page_source, encoding="utf-8")[0]
        else:
            dfs_temp = []
            for li in soup.find_all("div", "booklist"):
                href = url + li.find("h2").find("a")["href"]
                
                driver.get(href)
                click_more_btn(driver)
                dfs_temp.append(pd.read_html(driver.page_source, encoding="utf-8")[0])
            return pd.concat(tgt, axis=0, ignore_index=True)
    except:
        print(f"「{url}」無法爬取！")

In [3]:
# 搜尋目標
# ISBN = "9789869109321"  # 爬取＂宜蘭縣公共圖書館＂時，會有兩筆搜索結果
# title = "二十一世紀資本論"
# ISBN = "9789573317241"  # "哈利波特：神秘的魔法石"
# title = "哈利波特：神秘的魔法石"

# 設定 driver
my_options = Options()
my_options.add_argument("--incognito")  # 開啟無痕模式
# my_options.add_argument("--headless")  # 不開啟實體瀏覽器

capabilities = DesiredCapabilities.CHROME
capabilities["pageLoadStrategy"] = "none"

########## 初始化 df_DataFrame
ISBN = "9789573317241"
df_result = []

# 開啟 Chrome，設定等待元素最多 10 秒
driver = webdriver.Chrome(options=my_options, desired_capabilities=capabilities)
wait = WebDriverWait(driver, 10)

########## 初始化

In [4]:
# 搜尋目標
ISBN = "9789869109321"  # 爬取＂宜蘭縣公共圖書館＂時，會有兩筆搜索結果
# title = "二十一世紀資本論"
# ISBN = "9789573317241"  # "哈利波特：神秘的魔法石"
# title = "哈利波特：神秘的魔法石"

# 設定 driver
my_options = Options()
my_options.add_argument("--incognito")  # 開啟無痕模式
# my_options.add_argument("--headless")  # 不開啟實體瀏覽器

capabilities = DesiredCapabilities.CHROME
capabilities["pageLoadStrategy"] = "none"

########## 初始化 df_DataFrame
ISBN = "9789573317241"
df_result = []

# 開啟 Chrome，設定等待元素最多 10 秒
driver = webdriver.Chrome(options=my_options, desired_capabilities=capabilities)
wait = WebDriverWait(driver, 10)

########## 初始化



In [5]:
while True:
    print(
        webpac_gov_crawler("宜蘭縣公共圖書館", "https://webpac.ilccb.gov.tw/", ISBN,  driver, wait).shape[0]
    )

C:\ProgramData\Anaconda3\lib\site-packages\bs4\__init__.py:221: UserWarning: You provided Unicode markup but also provided a value for from_encoding. Your from_encoding will be ignored.
  warnings.warn("You provided Unicode markup but also provided a value for from_encoding. Your from_encoding will be ignored.")


「https://webpac.ilccb.gov.tw/」無法爬取！


AttributeError: 'NoneType' object has no attribute 'shape'